In [0]:
from pyspark.sql.functions import col, current_timestamp, lit, row_number
from pyspark.sql.window import Window

In [0]:
df_partidas = spark.table("project_data_football_bronze.partidas_rodada")

In [0]:
df_estadio = (
    df_partidas
        .select(col("local"))
        .filter(col("local").isNotNull())
        .dropDuplicates()
)

In [0]:
window_spec = Window.orderBy("local")

In [0]:
df_dim_estadio = (
    df_estadio
        .withColumn("estadio_id", row_number().over(window_spec))
        .withColumnRenamed("local", "nome_estadio")
        .withColumn("dt_ultima_atualizacao", current_timestamp())
)

In [0]:
df_dim_estadio.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("project_data_football_silver.dim_estadio")